In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [2]:
from dsgtools.azure import FileFisher
ff = FileFisher(
    storage_account_name="datalakestoragenfsawwsj9",
    container_name="datalakegen2filesystem",
)

In [3]:
filename_list = ff.finder().get('RnD Projects').get("Product RnD").get("Business").get("11657").get("Data Modeling").get("Returned").get("2105").to_list()
print(len(filename_list))

6


In [4]:
path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/"

In [5]:
name = "model2105"
keep = ["accountnumber", "LNLexIDSELE", "AssetPropCountEver", 'AssetPropAssessedTotalEver', 'AssetPropAssessedTotalCurrent', ]
keep = [x.lower() for x in keep]
for i, f in enumerate(filename_list):
    df = pd.read_adls(f, reader = pd.read_csv, compression = "gzip", encoding='iso-8859-1', dtype = str, verbose = False, usecols = keep)
    df.to_adls(path + "/_temp/" + name + "_" + str(i) + ".parquet", format = ".parquet", overwrite = True)
    if i == 0:
        stacked = df
    else:
        stacked = pd.concat([stacked, df], ignore_index = True)
    print(stacked.shape)
stacked.to_adls(path + "/_temp/" + name + "_all.parquet", format = ".parquet", overwrite = True)

(699972, 5)
(699977, 5)
(1399972, 5)
(2099972, 5)
(2799972, 5)
(3272546, 5)


In [17]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, np.inf], exceptions = [-1])
freq(stacked.assetpropcountever.astype(float), format = fmt)

,Count,Pct,Cuml Count,Cuml Pct
assetpropcountever,,,,
<= 0,1936039.0,0.591600,1936039.0,0.591600
1,353546.0,0.108034,2289585.0,0.699634
2,16094.0,0.004918,2305679.0,0.704552
3,3838.0,0.001173,2309517.0,0.705725
4,1240.0,0.000379,2310757.0,0.706104
5,487.0,0.000149,2311244.0,0.706253
6+,831.0,0.000254,2312075.0,0.706506
-1,959560.0,0.293215,3271635.0,0.999722
Missing,911.0,0.000278,3272546.0,1.000000


In [15]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, np.inf], exceptions = [-1])
freq(stacked.assetpropassessedtotalever.astype(float), format = fmt)

,Count,Pct,Cuml Count,Cuml Pct
assetpropassessedtotalever,,,,
<= 0,1954779.0,0.597327,1954779.0,0.597327
1,187.0,0.000057,1954966.0,0.597384
2,48.0,0.000015,1955014.0,0.597398
3,14.0,0.000004,1955028.0,0.597403
4,8.0,0.000002,1955036.0,0.597405
5,23.0,0.000007,1955059.0,0.597412
6+,357016.0,0.109094,2312075.0,0.706506
-1,959560.0,0.293215,3271635.0,0.999722
Missing,911.0,0.000278,3272546.0,1.000000


In [14]:
stacked.assetpropassessedtotalcurrent.astype(float).describe()

count    3.271635e+06
mean     9.532836e+04
std      2.535578e+06
min     -1.000000e+00
25%     -1.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+09
Name: assetpropassessedtotalcurrent, dtype: float64

In [8]:
fmt = make_format(cuts = [-np.inf, 0, np.inf], exceptions = [-1])
freq(stacked.assetpropcountever.astype(float), stacked.assetpropassessedtotalever.astype(float), format = [fmt, fmt], cross = False, observed = True)

Count       Pct  \
assetpropcountever assetpropassessedtotalever                        
<= 0               <= 0                        1936039.0  0.591600   
1+                 <= 0                          18740.0  0.005726   
                   1+                           357296.0  0.109180   
-1                 -1                           959560.0  0.293215   
Missing            Missing                         911.0  0.000278   

                                               Cuml Count  Cuml Pct  
assetpropcountever assetpropassessedtotalever                        
<= 0               <= 0                         1936039.0  0.591600  
1+                 <= 0                         1954779.0  0.597327  
                   1+                           2312075.0  0.706506  
-1                 -1                           3271635.0  0.999722  
Missing            Missing                      3272546.0  1.000000

Bad pipe message: %s [b'\x17+\x94\xc7\x1a@>\x0c\xe4\t\xce\x04\xef1\xfd2\xe47 \x8a8\x03\x86r\xf7\x93\x81El\x1c\xdb|\x1e>-|\xc0\r5\x95\xbc']
Bad pipe message: %s [b"\x86\x14>\xfb\xe4\xf8p\x01*\x1c\\\x87\xb7/\xb3\xc8\xcb\x98\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99"]
Bad pipe message: %s [b"l\x96H\xce*v\x16\xbfP}s\xa70u\xcf%y\x0f\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0\x13\x

In [7]:
stacked[stacked.assetpropassessedtotalcurrent.astype(float) > 0]

,accountnumber,lnlexidsele,assetpropcountever,assetpropassessedtotalever,assetpropassessedtotalcurrent
10,SBFEAccountOriginations0000013,1156,1,2630700,2630700
17,SBFEAccountOriginations0000020,1834,1,1305500,1305500
25,SBFEAccountOriginations0000029,2651,1,10166,10166
26,SBFEAccountOriginations0000030,2769,1,1810000,1810000
28,SBFEAccountOriginations0000032,2914,1,1250,1250
...,...,...,...,...,...
3269406,SBFEAccountOriginations3610319,139230876738,1,2474519,2474519
3270980,SBFEAccountOriginations3614918,139637076049,1,773800,773800
3271438,SBFEAccountOriginations3616053,139637491501,1,413360,413360
3272478,SBFEAccountOriginations3618171,139638898327,1,139688,139688


In [12]:
stacked.tail()

,accountnumber,lnlexidsele,assetpropassessedtotalever,assetpropassessedtotalcurrent
3272541,SBFEAccountOriginations3618322,139772896575,-1,-1
3272542,SBFEAccountOriginations3618325,139772970615,-1,-1
3272543,SBFEAccountOriginations3618327,139773117881,-1,-1
3272544,SBFEAccountOriginations3618331,139773227025,-1,-1
3272545,SBFEAccountOriginations3618335,134152795112,0,0
